# Builder (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.12/lib/netstandard2.1/NetMQ.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

## buildProject

In [ ]:
let buildProject path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let extension = fullPath |> System.IO.Path.GetExtension

    let getLocals () = $"fullPath: {fullPath} / {getLocals ()}"
    trace Debug (fun () -> "buildProject") getLocals

    match extension with
    | ".fsproj" -> ()
    | _ -> failwith "Invalid project file"

    let! exitCode, _result =
        Runtime.executeWithOptionsAsync
            {
                Command = "dotnet build -c Release"
                CancellationToken = None
                OnLine = None
                WorkingDirectory = Some fileDir
            }

    return exitCode
}

## buildCode

In [ ]:
let buildCode path name code = async {
    let getLocals () = $"path: {path} / name: {name} / code.Length: {code |> String.length} / {getLocals ()}"
    trace Debug (fun () -> "buildCode") getLocals

    let targetPath = path </> "target"
    System.IO.Directory.CreateDirectory targetPath |> ignore

    let filePath = targetPath </> $"{name}.fs" |> System.IO.Path.GetFullPath
    do! code |> FileSystem.writeAllTextAsync filePath

    let repositoryRoot = path |> FileSystem.findParent ".paket" false

    let fsprojPath = targetPath </> $"{name}.fsproj"
    let fsprojCode = $"""<Project Sdk="Microsoft.NET.Sdk">
    <PropertyGroup>
        <TargetFramework>net8.0</TargetFramework>
        <LangVersion>preview</LangVersion>
        <RollForward>Major</RollForward>
        <TargetLatestRuntimePatch>true</TargetLatestRuntimePatch>
        <Version>0.0.1-alpha.1</Version>
        <OutputType>Exe</OutputType>
    </PropertyGroup>

    <ItemGroup>
        <Compile Include="{repositoryRoot}/nbs/Common.fs" />
        <Compile Include="{repositoryRoot}/nbs/Async.fs" />
        <Compile Include="{repositoryRoot}/nbs/AsyncSeq.fs" />
        <Compile Include="{repositoryRoot}/nbs/Networking.fs" />
        <Compile Include="{repositoryRoot}/nbs/Runtime.fs" />
        <Compile Include="{repositoryRoot}/nbs/FileSystem.fs" />
        <Compile Include="{filePath}" />
    </ItemGroup>

    <Import Project="{repositoryRoot}/.paket/Paket.Restore.targets" />
</Project>
"""
    do! fsprojCode |> FileSystem.writeAllTextAsync fsprojPath

    let paketReferencesCode = $"FSharp.Core

Argu
FParsec
FSharp.Control.AsyncSeq
NetMQ
System.CommandLine
System.Reactive.Linq
"
    do! paketReferencesCode |> FileSystem.writeAllTextAsync (targetPath </> "paket.references")

    return! fsprojPath |> buildProject
}

In [ ]:
//// test

let tempFolder = FileSystem.getSourceDirectory () </> "target/test"
buildCode tempFolder "test" "1 + 1"
|> Async.runWithTimeoutStrict 20000
|> _equal (Some 0)

00:00:00 #1 [Debug] buildCode / path: C:\home\git\polyglot\apps\builder\target/test / name: test / code.Length: 5
00:00:00 #2 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj
00:00:00 #3 [Debug] executeAsync / options: { Command = "dotnet build -c Release"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target\test\target"
  CancellationToken = None
  OnLine = None }
00:00:00 #4 [Debug] > MSBuild version 17.7.0+5785ed5c2 for .NET
00:00:01 #5 [Debug] >   Determining projects to restore...
00:00:02 #6 [Debug] >   All projects are up-to-date for restore.
00:00:02 #7 [Debug] > C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\8.0.100-preview.6.23330.14\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(314,5): message NETSDK1057: You are using a preview version of .NET. See: https://aka.ms/dotnet-support-policy [C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj]
00:00:06 #8 [Deb

In [ ]:
//// test

let tempFolder = FileSystem.getSourceDirectory () </> "target/test"
buildCode tempFolder "test" "1 + a"
|> Async.runWithTimeoutStrict 20000
|> _equal (Some 1)

00:00:09 #21 [Debug] buildCode / path: C:\home\git\polyglot\apps\builder\target/test / name: test / code.Length: 5
00:00:09 #22 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj
00:00:09 #23 [Debug] executeAsync / options: { Command = "dotnet build -c Release"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target\test\target"
  CancellationToken = None
  OnLine = None }
00:00:09 #24 [Debug] > MSBuild version 17.7.0+5785ed5c2 for .NET
00:00:10 #25 [Debug] >   Determining projects to restore...
00:00:10 #26 [Debug] >   All projects are up-to-date for restore.
00:00:10 #27 [Debug] > C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\8.0.100-preview.6.23330.14\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(314,5): message NETSDK1057: You are using a preview version of .NET. See: https://aka.ms/dotnet-support-policy [C:\home\git\polyglot\apps\builder\target\test\target\test.fsproj]
00:00:13 

## buildFile

In [ ]:
let buildFile path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let dir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    return! code |> buildCode dir fileName
}

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce; Argu.ArguAttributes.Last>]
        Paths of paths : string list

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Paths _ -> nameof Arguments.Paths

## main

In [ ]:

let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let paths =
        match argsMap.[nameof Arguments.Paths] with
        | [ Arguments.Paths paths ] -> paths
        | _ -> []

    paths
    |> List.map buildFile
    |> Async.Parallel
    |> Async.runWithTimeout 30000
    |> function
        | Some results -> results |> Array.sum
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0

00:00:13 #38 [Debug] buildCode / path: C:\home\git\polyglot\apps\builder / name: Builder / code.Length: 4014
00:00:13 #39 [Debug] buildProject / fullPath: C:\home\git\polyglot\apps\builder\target\Builder.fsproj
00:00:13 #40 [Debug] executeAsync / options: { Command = "dotnet build -c Release"
  WorkingDirectory = Some "C:\home\git\polyglot\apps\builder\target"
  CancellationToken = None
  OnLine = None }
00:00:13 #41 [Debug] > MSBuild version 17.7.0+5785ed5c2 for .NET
00:00:14 #42 [Debug] >   Determining projects to restore...
00:00:15 #43 [Debug] >   All projects are up-to-date for restore.
00:00:15 #44 [Debug] > C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\8.0.100-preview.6.23330.14\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(314,5): message NETSDK1057: You are using a preview version of .NET. See: https://aka.ms/dotnet-support-policy [C:\home\git\polyglot\apps\builder\target\Builder.fsproj]
00:00:20 #45 [Debug] >   Builder -> C:\home\g